In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [3]:
# 📦 Step 1: Load datasets
import pandas as pd
import numpy as np
import time
import tensorflow as tf

print("🔎 TensorFlow version:", tf.__version__)

# Force CPU strategy
print("💻 Running on CPU!")
strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")

# Load datasets
raw_df = pd.read_csv('/content/drive/MyDrive/natural_disasters_2024.csv')
geo_df = pd.read_csv('/content/drive/MyDrive/cleaned_disasters.csv')
print("✅ Datasets loaded from Google Drive")

# 🛠️ Step 2: Preprocess
raw_df.columns = raw_df.columns.str.strip()
geo_df.columns = geo_df.columns.str.strip()

# Ensure float types
for col in ['Latitude', 'Longitude', 'Magnitude', 'Severity']:
    if col in geo_df.columns:
        geo_df[col] = geo_df[col].astype(np.float32)
    if col in raw_df.columns:
        raw_df[col] = raw_df[col].astype(np.float32)

print("⚙️ Pandas DataFrames ready")

# 🧠 Step 3: Train TensorFlow DNN
X_train = geo_df[['Latitude', 'Longitude', 'Magnitude']].values
y_train = geo_df['Severity'].values

with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation='relu', input_shape=(3,)),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1)  # Regression output
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1)

model.save('trained_dnn_model.keras')
print("✅ TensorFlow DNN model trained and saved")

# 📡 Step 4: Simulate streaming data
# Randomly generate Latitude, Longitude without cupy (using numpy)
stream_batch = raw_df.sample(n=10000).reset_index(drop=True)

np.random.seed(42)
stream_batch['Latitude'] = np.random.uniform(-90, 90, size=len(stream_batch))
stream_batch['Longitude'] = np.random.uniform(-180, 180, size=len(stream_batch))
stream_batch['Magnitude'] = stream_batch['Magnitude'].astype(np.float32)

X_stream = stream_batch[['Latitude', 'Longitude', 'Magnitude']].values

records = stream_batch.to_dict(orient='records')
success_count = 0
failure_count = 0
send_times = []
total_bytes_sent = 0

start_time = time.time()

for idx, record in enumerate(records):
    try:
        # Predict Severity
        features = np.array([[record['Latitude'], record['Longitude'], record['Magnitude']]])
        pred = model.predict(features, verbose=0)[0][0]
        record['Predicted_Severity'] = float(pred)

        # Simulate sending
        t0 = time.time()
        size = len(str(record).encode('utf-8'))
        time.sleep(0.01)  # Artificial latency
        t1 = time.time()

        send_times.append(t1 - t0)
        total_bytes_sent += size
        success_count += 1
    except Exception as e:
        failure_count += 1

end_time = time.time()

# ⏱️ Step 5: Calculate Metrics
total_time = end_time - start_time
avg_speed = success_count / total_time if total_time > 0 else 0
throughput = total_bytes_sent / total_time if total_time > 0 else 0
min_send = min(send_times) if send_times else 0
max_send = max(send_times) if send_times else 0
mean_send = np.mean(send_times) if send_times else 0
median_send = np.median(send_times) if send_times else 0
std_send = np.std(send_times) if send_times else 0

# 📢 Step 6: Print Metrics
print(f"🔢 Total Records: {len(records)}")
print(f"✅ Successfully Sent: {success_count}")
print(f"❌ Failed Sends: {failure_count}")
print(f"⏱️ Total Time Taken: {total_time:.2f} seconds")
print(f"⚡ Average Send Speed: {avg_speed:.2f} records/second")
print(f"📦 Total Data Sent: {total_bytes_sent / 1024:.2f} KB")
print(f"🚀 Throughput: {throughput:.2f} bytes/second")
print(f"📈 Min Send Time: {min_send:.4f} sec")
print(f"📉 Max Send Time: {max_send:.4f} sec")
print(f"📊 Mean Send Time: {mean_send:.4f} sec")
print(f"📏 Median Send Time: {median_send:.4f} sec")
print(f"📐 Std Dev Send Time: {std_send:.4f} sec")

# 🚨 Step 7: Save critical alerts
final_df = pd.DataFrame(records)
alerts = final_df[final_df['Predicted_Severity'] > 8]

if len(alerts) > 0:
    alerts_path = "/content/critical_alerts.csv"
    alerts.to_csv(alerts_path, index=False)
    print(f"⚠️ Critical alerts saved to {alerts_path}")
    from google.colab import files
    files.download(alerts_path)
else:
    print("✅ No critical alerts found.")


🔎 TensorFlow version: 2.18.0
💻 Running on CPU!
✅ Datasets loaded from Google Drive
⚙️ Pandas DataFrames ready


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 25.0128
Epoch 2/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6882
Epoch 3/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.6255
Epoch 4/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6379
Epoch 5/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.5940
Epoch 6/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.6601
Epoch 7/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.6602
Epoch 8/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6286
Epoch 9/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.7172
Epoch 10/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6722
Epoch 11/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6125
Epoch 12/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6317
Epoch 13/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6359
Epoch 14/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.6061
Epoch 15/20
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - l

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>